<a href="https://colab.research.google.com/github/roesta07/How-Business-Strategies-can-create-bias/blob/main/strategy_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports and utility functions
# Install packages that are not installed in colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install pymc3==3.9
    !pip install --upgrade daft
    !pip install --upgrade theano
    !pip install causalgraphicalmodels
    !pip install watermark
    !pip install arviz --no-dependencies
    !pip install netCDF4
    !pip install xarray

Requirement already up-to-date: daft in /usr/local/lib/python3.6/dist-packages (0.1.0)
Requirement already up-to-date: theano in /usr/local/lib/python3.6/dist-packages (1.0.5)


In [2]:
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import daft
from theano import theano as shared

from scipy.interpolate import griddata
from causalgraphicalmodels import CausalGraphicalModel

In [3]:
%config InlineBackend.figure_format = 'retina'
az.style.use('arviz-darkgrid')
# %load_ext watermark
az.rcParams["stats.hdi_prob"] = 0.89  # sets default credible interval used by arviz€

In [25]:
## business simulation
n_samples=500
M=np.random.normal(10,2,size=n_samples)
## which are regeions
## east or west
U = np.random.binomial(1,0.3,size=n_samples)
O = np.random.normal(M-(U*3),1,size=n_samples)
P = np.random.normal(B*5-(U*2),1,size=n_samples)

ValueError: ignored

In [17]:
## lets create a dataframe
df=pd.DataFrame(np.vstack([M,O,P,U]).T,columns=['M','O','P','U'])
df.head()

,M,O,P,U
0,5.375884,2.040841,32.893216,1.0
1,6.300895,5.462063,50.118948,0.0
2,9.425974,5.128543,44.241558,1.0
3,9.355080,9.639476,52.398091,0.0
4,11.149438,9.327295,71.205543,0.0


In [18]:
## Some number transformations

def std_standarize(col):
  '''Takes in a numpy array, returns its standard unit'''
  return (col-col.mean())/col.std()

def normalize(col):
  '''Takes in a numpy array, returns min-max scale'''
  return col/col.max()

df=df.assign(M_std=standarize(df['M']),O_norm=normalize(df['O']),P_std=standarize(df['P']))


In [21]:
## Modeling

with pm.Model() as m_1:
  a=pm.Normal('a',0,0.2)
  bM=pm.Normal('bM',0,0.5)
  bO=pm.Normal('bO',0,0.5)
  sigma=pm.Exponential('sigma',1)
  mu=pm.Deterministic('mu',a+bM*(df['M_std'])+bO*(df['O_norm']))
  profit=pm.Normal('profit',mu=mu,sigma=sigma,observed=df['P_std'])
  trace_1=pm.sample()



Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, bO, bM, a]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.


In [23]:
az.summary(trace_1,var_names=['~mu'])

/usr/local/lib/python3.6/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a,-0.101,0.162,-0.350,0.170,0.005,0.004,999.0,978.0,1001.0,1094.0,1.0
bM,-0.043,0.085,-0.169,0.096,0.003,0.002,1006.0,968.0,1003.0,1157.0,1.0
bO,0.178,0.258,-0.264,0.561,0.008,0.006,934.0,934.0,936.0,1104.0,1.0
sigma,1.003,0.053,0.920,1.091,0.001,0.001,1894.0,1894.0,1881.0,1250.0,1.0
